## Basic Flow

#### This notebook describes a basic flow of ml-git with api

#### To start using the ml-git api we need to import it into our script

In [ ]:
from ml_git import api

#### After that, we define some variables that will be used by the notebook

In [ ]:
# The type of entity we are working on
entity = 'dataset'

# The entity name we are working on
entity_name = 'ms'

#### To start, let's take into account that you have a repository with git settings and you want to make the clone. If this is not your scenario, you will need to configure ml-git outside this notebook (At the moment the api does not have the necessary methods to perform this configuration)

In [ ]:
repository_url = 'https://git@github.com/mlgit-repository'

api.clone(repository_url)

#### When the clone is successfully completed, the entities are initialized and ready for use. We assume there is an existing ml-git repository with a few ML datasets under its management and you'd like to download one of the existing datasets. 

#### Now, it is quite simple to retrieve a specific version locally to start any experiment with:

In [ ]:
# Version of the dataset you want to checkout
tag = 'computer-vision__images__mscoco__1'

data_path = api.checkout(entity, tag)

#### To continue with this notebook it will be necessary to add new data to the entity that we are working on. For this, the following code generate a new file in our dataset path.

In [ ]:
import os

file_path = os.path.join(data_path, 'data', 'new_file')
open(file_path, 'a').close()

#### We can now proceed with the necessary steps to send the new data to storage.

In [ ]:
api.add('dataset', 'mscoco', bumpversion=True)

Commit the changes

In [ ]:
# Custom commit message
message = 'Commit example'

api.commit(entity, entity_name, message)

And finally, sending the data

In [ ]:
api.push(entity, entity_name)